<a href="https://colab.research.google.com/github/cqzw555/Colaboratory/blob/main/cifar10_resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# 使用预训练模型
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import os
from torchsummary import summary
from torchvision.models import resnet18

# 获取数据
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomGrayscale(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)
transform1 = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 512
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform1)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

model = resnet18(pretrained=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.fc = nn.Linear(512, 10)
model = model.to(device)
model.to(device)
summary(model, (3, 32, 32))

Files already downloaded and verified
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 

In [ ]:
epoch_num = 30
# 分类问题一般用交叉熵
criterion = nn.CrossEntropyLoss()
# 使用随机梯度下降，L2正则化
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
timestr = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())

for epoch in range(0, epoch_num):
  # 训练
  loss_sum = 0.0
  correct = 0.0
  total = 0.0
  length = len(trainloader)
  for index, (data, label) in enumerate(trainloader):
    data, label = data.to(device), label.to(device)
    optimizer.zero_grad()
    # forward and backward
    output = model(data)
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()
    # 打印输出信息
    loss_sum += loss.item()
    _, pred = torch.max(output.data, 1)
    total += label.size(0)
    correct += pred.eq(label.data).sum()
    if index % 50 == 0:
        log = "[epoch :{:d}] {}/{} Loss: {:.1f} Acc:{:.1f}".format(
            epoch, index * len(data), len(trainloader.dataset), loss_sum / (index + 1),
                    100.0 * correct / total)
        # 打印log
        print(log)

  # 测试
  with torch.no_grad():
    correct = 0
    total = 0
    for data, label in testloader:
        model.eval()
        data, label = data.to(device), label.to(device)
        output = model(data)
        _, pred = torch.max(output.data, 1)
        total += label.size(0)
        correct += (pred == label).sum()
    acc = 100.0 * correct / total
    # torch.save(model.state_dict(), './model/{}/{}/{}.pth'.format(model_name, timestr, epoch))
    log = '[Test {}] Acc:{:.1f}'.format(epoch, acc)
    print(log)

[epoch :0] 0/50000 Loss: 2.7 Acc:9.4
[epoch :0] 25600/50000 Loss: 1.3 Acc:53.6
[epoch :1] 0/50000 Loss: 0.6 Acc:78.7
[epoch :1] 25600/50000 Loss: 0.8 Acc:72.6
[Test 1] Acc:77.3
[epoch :2] 0/50000 Loss: 0.6 Acc:77.9
[epoch :2] 25600/50000 Loss: 0.6 Acc:78.4
[Test 2] Acc:78.1
[epoch :3] 0/50000 Loss: 0.6 Acc:79.5
[epoch :3] 25600/50000 Loss: 0.5 Acc:81.1
[Test 3] Acc:79.9
[epoch :4] 0/50000 Loss: 0.4 Acc:85.2
[epoch :4] 25600/50000 Loss: 0.5 Acc:83.5
[Test 4] Acc:80.7
[epoch :5] 0/50000 Loss: 0.5 Acc:83.6
[epoch :5] 25600/50000 Loss: 0.4 Acc:84.5
[Test 5] Acc:80.6
[epoch :6] 0/50000 Loss: 0.4 Acc:85.4
[epoch :6] 25600/50000 Loss: 0.4 Acc:86.7
[Test 6] Acc:82.6
[epoch :7] 0/50000 Loss: 0.3 Acc:89.8
[epoch :7] 25600/50000 Loss: 0.3 Acc:88.1
[Test 7] Acc:82.1
[epoch :8] 0/50000 Loss: 0.3 Acc:89.6


In [ ]:
torch.save(model.state_dict(),'cifar10_resNet18_colab.pth')

In [ ]:
! scp cifar10_resNet18_colab.pth ubuntu@121.4.124.199:~/

Host key verification failed.
lost connection


In [ ]:
!sshpass -p zW112559 scp -P 22 cifar10_resNet18_colab.pth ubuntu@121.4.124.199:/home/ubuntu/cifar10_resNet18_colab.pth